## Tokenisation


In [14]:
import urllib.request
import re
import tiktoken
from gptlight.datasets import fetch_verdict_text
from gptlight.tokenizer import SimpleTokenizer

### Récupérer les données


In [2]:
raw_text = fetch_verdict_text()
print(f"Total number of character : {len(raw_text)}")

Total number of character : 20479


- Tokenisation de base


In [3]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [5]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- Vocabulaire


In [6]:
all_tokens = sorted(set(preprocessed))

vocabulary = {token:token_id for token_id, token in enumerate(all_tokens)}

In [7]:
print(f"Number of tokens : {len(all_tokens)}\n")
for i, item in enumerate(vocabulary.items()):
    print(item)
    
    if i >= 50:
        break


Number of tokens : 1130

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


La classe Python `SimpleTokenizerV1` permet d'instancier de nouveaux tokenizers à partir d'un vocabulaire existant, que l'on peut ensuite utiliser pour encoder et décoder du texte.


In [7]:
tokenizer = SimpleTokenizer(vocabulary)

In [8]:
text = """"It's the last he painted, you know,
Mrs. Gisburn said with pardonable pride."""

tokens_ids= tokenizer.encode(text)
print(tokens_ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 7, 38, 851, 1108, 754, 793, 7]


In [9]:
print(tokenizer.decode(tokens_ids))

" It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride.


- Ajouter des tokens de contexte spéciaux

Nous ajoutons des tokens spéciaux à un vocabulaire pour gérer certains contextes. Par exemple, nous introduisons un token <|unk|> afin de représenter les mots nouveaux ou inconnus qui n'étaient pas présents dans les données d'entraînement et donc pas dans le vocabulaire existant. De plus, nous ajoutons un token <|endoftext|> que l'on peut utiliser pour séparer deux sources de texte indépendantes.


In [10]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocabulary = {token:token_id for token_id,token in enumerate(all_tokens)}

In [11]:
tokenizer = SimpleTokenizer(vocabulary)

In [12]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))

tokens_ids = tokenizer.encode(text)
print(tokens_ids)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [13]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


- Encodage par paires de bytes avec le module open source tiktoken


In [15]:
tokenizer = tiktoken.get_encoding("gpt2")

In [20]:
text = (
"Hello, do you like tea? <|endoftext|> In the sunlit terraces"
"of someunknownPlace."
)

In [21]:
tokens_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(tokens_ids)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [22]:
tokenizer.decode(tokens_ids)

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'